In [1]:
import rasterio
import rasterio.plot
import numpy as np
from osgeo import gdal_array
from scipy.signal import savgol_filter
from scipy.interpolate import pchip_interpolate, InterpolatedUnivariateSpline
import math
from scipy.optimize import minimize
from scipy.optimize import Bounds
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def double_logistic_array(params,timeArr):
    phenology=[]
    for time in timeArr:
        phenology.append(double_logistic(params,time))
    return np.array(phenology)

def double_logistic(params,time):
    """Generate a double logistic curve similar 
    to those of the MODIS phenology product
    
    k1   --- curvature parmeter for first half of season
    k2   --- curvature parmeter for second half of season
    t01  --- timing parmeter for first half of season
    t02  --- timing parmeter for second half of season
    c    --- minimum LAI/NDVI etc
    d    --- maximum LAI/NDVI etc
    
    time --- time valueto evaluate the function 
    """
    k1  =params[0]
    k2  =params[1]
    t01 =params[2]
    t02 =params[3]
    c   =params[4]
    d   =params[5]
    lgstc1=(c-d)/(1.+math.exp(k1*(time-t01)))+d    
    lgstc2=(d-c)/(1.+math.exp(k2*(time-t02)))+c
    return np.min([lgstc1,lgstc2])
    
def genSynthObs(params,freq=1,stddev=0.0):
    
    time_arr=[]
    synth_obs=[]
    for t in range(1,365,freq):
        time_arr.append(t)
        obs=double_logistic(params,t)
        synth_obs.append(obs)
        
    return np.array(synth_obs),np.array(time_arr)

def phenology_differential(synth_obs):
    differential = []
    for i in range(len(synth_obs) -1):
        differential.append(synth_obs[i+1] - synth_obs[i])
    sos = differential.index(max(differential)) +1
    eos = differential.index(min(differential)) +1s
    return sos, eos

def rmse(params,obs,obs_time):
    model=double_logistic_array(params,obs_time)
    rmse_series = np.sqrt(np.sum((model-obs)**2)/float(len(obs_time)))
    return rmse_series
    
def solver(params,obs,obs_time,costFunction,bounds):
    #costGradient, costHessian=ad.gh(costFunction)
    return minimize(costFunction,\
                    params,\
                    args=(obs,obs_time),\
                    bounds=bounds,\
                    #method='L-BFGS-B',\
                    jac='2-point',\
                    options={'gtol': 1e-6})

In [ ]:
# 读取滤波后的影像
param_img_path = r'data/2019_params.tif'
param_img = gdal_array.LoadFile(param_img_path)
[doy, img_ht, img_wd] = param_img.shape              # 影像的维度（日期数、宽、高）
param_num = 6       # double logistic 参数个数

sos_img = np.zeros((img_ht, img_wd))
eos_img = np.zeros((img_ht, img_wd))

# 分配影像日期(DOY)
time_arr=np.arange(1, 365, 16)

# 开始计时
start = time.time()
for x in range(1,img_ht):
    # 打印进度信息，因为影像较大。
    if x%100 == 0:
        end = time.time()
        print(x, ' of ', img_ht, '\t', x/img_ht *100, '%\t Time elapsed: ', str(end - start) )
        
    for y in range(1,img_wd):
        params = param_img[:,x,y] # 获取拟合参数
        series = genSynthObs(params)

        # 如果序列全为 0，则不需要做拟合（研究区外）
        if np.count_nonzero(series) > 3:

            sos_img[x,y], eos_img[x,y] = phenology_differential(series)
